# Asimetrična šifra RSA

## Naloga 4

Implementirajte asimetrično šifro RSA, tako da podate implementacijo algoritmov za ustvarjanje ključev, šifriranja in dešifriranja. Pri tem si pomagajte z dokumentacijo knjižnice cryptography: [RSA @ cryptography](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/)

Specifikacije so naslednje:

Algoritem `gen(int)`
- opcijsko vzame velikost ključa (modula); privzeta vrednost naj bo 2048 bitov;
- eksponent e naj bo 65537
- vrne naj par `(pk, sk)`

In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa
def gen(key_size=2048):
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=None)
    public_key = private_key.public_key()
    return public_key, private_key

In [2]:
def test_gen():
    pk, sk = gen()
    assert hasattr(pk, 'encrypt')
    assert hasattr(pk, 'public_bytes')
    assert hasattr(sk, 'decrypt')
    assert hasattr(sk, 'sign')
    assert hasattr(sk, 'private_bytes')
    
test_gen()

Šifrirni algoritem vzame tri argumente:
- `pk` javni ključ kot ga vrne algoritem `gen()`
- `ct` čistopis v bajtih
- opcijsko `aad`, pridruženi podatki v bajtih

Za podlaganje uporabite algoritem `OAEP`, za `MGF` algoritem `MGF1` z `SHA256` in za zgoščevalno funkcijo prav tako `SHA256`.

Kot rezultat vrnite bajte tajnopisa.

In [3]:
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
def enc(pk, message: bytes, aad: bytes=None) -> bytes:
    ct = pk.encrypt(message, padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=aad))
    return ct

Dešifrirani algoritem vzame tri argumente:
- `sk` zasebni ključ kot ga vrne algoritem gen(),
- `ct` tajnopis v bajtih,
- opcijsko `aad`, pridruženi podatki v bajtih.

Podobno kot pri šifriranju: za podlaganje uporabite algoritem `OAEP`, za `MGF` algoritem `MGF1` z `SHA256` in za zgoščevalno funkcijo prav tako `SHA256`.

Kot rezultat vrnite bajte čistopisa oz. napako, če je tajnopis neveljaven.

In [4]:
def dec(sk, ct: bytes, aad: bytes=None) -> bytes:
    pt = sk.decrypt(ct, padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=aad))
    return pt


In [5]:
def test_enc_dec():
    pk, sk = gen()
    
    assert dec(sk, enc(pk, b"hello")) == b"hello"
    assert dec(sk, enc(pk, b"hello", b"world"), b"world") == b"hello"
    try:
        dec(sk, enc(pk, b"hello", b"world"))
        raise AssertionError
    except ValueError:
        pass
    
test_enc_dec()